In [ ]:
# Install TFOD API (TF 1)
%tensorflow_version 1.x
import tensorflow as tf 
print(tf.__version__)

!git clone https://github.com/tensorflow/models.git

% cd models/research
!pip install --upgrade pip
# Compile protos.
!protoc object_detection/protos/*.proto --python_out=.
# Install TensorFlow Object Detection API.
!cp object_detection/packages/tf1/setup.py .
!python -m pip install --use-feature=2020-resolver .

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [7]:
from imutils import paths
from tqdm.notebook import tqdm
import matplotlib.pyplot as plt
import numpy as np
import json 

In [8]:
def parse_image(image_path: str) -> np.ndarray:
    """Reads an image and adds a batch dimension."""
    image = plt.imread(image_path).astype(np.uint8)
    image = np.expand_dims(image, 0)
    return image

## Load test image paths

In [9]:
test_image_paths = list(paths.list_images("ShelfImages/test"))
test_image_paths[:5]

['ShelfImages/test/C2_P03_N2_S3_1.JPG',
 'ShelfImages/test/C1_P10_N1_S5_1.JPG',
 'ShelfImages/test/C3_P01_N2_S3_2.JPG',
 'ShelfImages/test/C4_P03_N1_S3_1.JPG',
 'ShelfImages/test/C2_P04_N3_S2_1.JPG']

## Load detection graph

In [10]:
detection_graph = tf.Graph()
with detection_graph.as_default():
    od_graph_def = tf.GraphDef()
    with tf.gfile.GFile("frozen_inference_graph.pb", "rb") as fid:
        serialized_graph = fid.read()
        od_graph_def.ParseFromString(serialized_graph)
        tf.import_graph_def(od_graph_def, name='')

## Inference utility

In [11]:
def run_inference_for_single_image(image, graph, min_threshold=0.6):
    """Runs detection graph on an image and parses the results."""
    with graph.as_default():
        with tf.Session() as sess:
            # Get handles to input and output tensors
            ops = tf.get_default_graph().get_operations()
            all_tensor_names = {output.name for op in ops for output in op.outputs}
            tensor_dict = {}
            for key in [
                "num_detections", "detection_boxes", "detection_scores",
                "detection_classes"]:
                tensor_name = key + ":0"
                if tensor_name in all_tensor_names:
                    tensor_dict[key] = tf.get_default_graph().get_tensor_by_name(
                        tensor_name)
                image_tensor = tf.get_default_graph().get_tensor_by_name("image_tensor:0")

            # Run inference
            output_dict = sess.run(tensor_dict,
                                    feed_dict={image_tensor: image})

    # Post-process the results
    output_dict["detection_scores"] = output_dict["detection_scores"][0]
    mask = output_dict["detection_scores"] > min_threshold

    return output_dict["detection_scores"][mask]

## Run bulk inference and prepare JSON file

In [12]:
image_to_products = {}
for image_path in tqdm(test_image_paths):
    image_name = image_path.split("/")[-1]
    image = parse_image(image_path)
    num_products = len(run_inference_for_single_image(image, detection_graph))
    image_to_products[image_name] = num_products

json_string = json.dumps(image_to_products, indent=4) 
with open("image2products.json", "w") as outfile: 
    outfile.write(json_string) 

In [13]:
!head -10 image2products.json

{
    "C2_P03_N2_S3_1.JPG": 22,
    "C1_P10_N1_S5_1.JPG": 51,
    "C3_P01_N2_S3_2.JPG": 17,
    "C4_P03_N1_S3_1.JPG": 33,
    "C2_P04_N3_S2_1.JPG": 21,
    "C3_P03_N2_S4_1.JPG": 49,
    "C3_P04_N1_S5_1.JPG": 40,
    "C4_P08_N2_S2_1.JPG": 24,
    "C4_P03_N1_S4_1.JPG": 45,


## References
* https://github.com/anirbankonar123/CorrosionDetector/blob/master/rust_localization.ipynb

https://www.youtube.com/playlist?list=PL1GQaVhO4f_jLxOokW7CS5kY_J1t1T17S



